In [ ]:
# This cell contains some utility functions to prepare and execute the benchmarks
import timeit
from statistics import median
from random import choice
from string import ascii_uppercase

def random_string(length):
    """Produce a random string made of *length* uppercase ascii characters"""
    return ''.join(choice(ascii_uppercase) for i in range(length))

def print_scaling(stmt, setup, sizes=[10000, 20000, 30000], repeat=False, units='us'):
    """Print scaling information for the statement *stmt*, executed after *setup*.
    
    The *setup* and *stmt* arguments take a template string where "{N}"
    will be replaced as the size of the input.
    
    The *repeat* flags determined if the setup needs to be run between
    each test run.
    """
    values = []
    for size in sizes:
        if repeat:
            timings = timeit.repeat(stmt.format(N=size),
                                    setup=setup.format(N=size),
                                    number=1, repeat=1000)
            values.append(min(timings))
        else:
            timings = timeit.repeat(stmt.format(N=size),
                                    setup=setup.format(N=size),
                                    number=1000, repeat=3)
            values.append(min(t/1000 for t in timings))
    unit_factor = {'us': 1e6,
                   'ms': 1e3}[units]
    
    print(' | '.join('N = {} t = {:.2f} ({})'.format(n, t * unit_factor, units) for n, t in zip(sizes, values)))

In [ ]:
print_scaling('collection.pop()',
              setup='collection = list(range({N}))')

print_scaling('collection.pop(0)',
              setup='collection = list(range({N}))')

print_scaling('collection.append(1)',
                        setup='collection = list(range({N}))')

print_scaling('collection.insert(0, 1)',
              setup='collection = list(range({N}))')

print_scaling('collection.insert(5000, 1)',
              setup='collection = list(range({N}))')

N = 10000 t = 0.02 (us) | N = 20000 t = 0.02 (us) | N = 30000 t = 0.02 (us)
N = 10000 t = 1.33 (us) | N = 20000 t = 2.29 (us) | N = 30000 t = 4.08 (us)
N = 10000 t = 0.03 (us) | N = 20000 t = 0.02 (us) | N = 30000 t = 0.02 (us)
N = 10000 t = 2.07 (us) | N = 20000 t = 3.52 (us) | N = 30000 t = 5.24 (us)
N = 10000 t = 0.91 (us) | N = 20000 t = 2.54 (us) | N = 30000 t = 4.19 (us)


In [ ]:
setup_code = '''
import random
random.seed(42)

collection = list(range({N}))
'''
print_scaling('collection.index(random.randint(0, {N}))',
              setup=setup_code)

N = 10000 t = 16.94 (us) | N = 20000 t = 26.53 (us) | N = 30000 t = 40.39 (us)


In [ ]:
from bisect import bisect_left

def index_bisect(a, x):
    'Locate the leftmost value exactly equal to x'
    i = bisect_left(a, x)
    if i != len(a) and a[i] == x:
        return i
    raise ValueError

setup_code = '''
from __main__ import index_bisect

import random
random.seed(42)

collection = list(range({N}))
'''
    
print_scaling('index_bisect(collection, random.randint(0, {N}))',
              setup=setup_code)

N = 10000 t = 0.58 (us) | N = 20000 t = 1.13 (us) | N = 30000 t = 1.19 (us)


In [ ]:
from collections import deque
import timeit

# Define the access operations to measure for deque
access_operations = {
    'a': 'deq[0]',
    'b': 'deq[-1]',
    'c': 'deq[int(N/2)]'  # Accessing the middle element
}

# List sizes to test
sizes = [10000, 20000, 30000]

# Perform the measurements for deque access operations
for size in sizes:
    setup_code = f"from collections import deque\nN = {size}\ndeq = deque(range(N))"

    print(f'Timing for deque access with {size} elements:')
    
    # Measure access time for each operation
    for op, code in access_operations.items():
        time_taken = timeit.timeit(stmt=code, setup=setup_code, number=10000)
        print(f'{op}. {code}: {time_taken / 10000:.6f} seconds')
    
    print('---')

Timing for deque access with 10000 elements:
a. deq[0]: 0.000000 seconds
b. deq[-1]: 0.000000 seconds
c. deq[int(N/2)]: 0.000000 seconds
---
Timing for deque access with 20000 elements:
a. deq[0]: 0.000000 seconds
b. deq[-1]: 0.000000 seconds
c. deq[int(N/2)]: 0.000000 seconds
---
Timing for deque access with 30000 elements:
a. deq[0]: 0.000000 seconds
b. deq[-1]: 0.000000 seconds
c. deq[int(N/2)]: 0.000000 seconds
---


In [ ]:
print_scaling('collection.pop()',
              setup='from collections import deque; collection = deque(range({N}))')


print_scaling('collection.popleft()',
              setup='from collections import deque; collection = deque(range({N}))')


print_scaling('collection.append(1)',
              setup='from collections import deque; collection = deque(range({N}))')

print_scaling('collection.appendleft(1)',
              setup='from collections import deque; collection = deque(range({N}))')

N = 10000 t = 0.02 (us) | N = 20000 t = 0.02 (us) | N = 30000 t = 0.02 (us)
N = 10000 t = 0.02 (us) | N = 20000 t = 0.02 (us) | N = 30000 t = 0.02 (us)
N = 10000 t = 0.02 (us) | N = 20000 t = 0.03 (us) | N = 30000 t = 0.03 (us)
N = 10000 t = 0.03 (us) | N = 20000 t = 0.02 (us) | N = 30000 t = 0.02 (us)


In [ ]:
print_scaling('collection[0]',
              setup='from collections import deque; collection = deque(range({N}))')
print_scaling('collection[{N} - 1]',
              setup='from collections import deque; collection = deque(range({N}))')
print_scaling('collection[int({N}/2)]',
              setup='from collections import deque; collection = deque(range({N}))')

N = 10000 t = 0.01 (us) | N = 20000 t = 0.01 (us) | N = 30000 t = 0.01 (us)
N = 10000 t = 0.01 (us) | N = 20000 t = 0.01 (us) | N = 30000 t = 0.01 (us)
N = 10000 t = 0.14 (us) | N = 20000 t = 0.23 (us) | N = 30000 t = 0.30 (us)
